# First-level GLM using Nipype FSL

In this notebook, we recreate the first-level GLM and the first level GLM of FSL GUI using nipype code. For each nipype node, we list the corresponding fsl command from the log file. The dataset we use is a Flanker task, which can be downloaded [here](https://openneuro.org/datasets/ds000102/versions/00001).

We also borrow some helps from this [document](https://nipype.readthedocs.io/en/latest/users/examples/fmri_fsl.html). 

**12/05/2021**
We modified two parts:
1. simplified the registration part - now it is the same as FSL command
2. save all first-level outputs into single/separate folder for further analysis use.

## Preparation
Import all the relevant libraries needed for the preprocessing stage.

In [ ]:
from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range

import os, stat  # system functions
import getpass
from glob import glob

from nipype import Function
import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.modelgen as model  # model generation
import nipype.algorithms.rapidart as ra  # artifact detection

%autosave 5

Set up data path

In [ ]:
# Get current user
user = getpass.getuser()
print('Running code as: ', user)

# Set the BIDS path
data_dir = '/home/{}/data/'.format(user)
# Set path where nipype will store stepwise results (e.g., masks)
exp_dir = '/home/{}/out/fsl/hw2/'.format(user)

try:
    os.mkdir(exp_dir)
except OSError as error:
    print(error)
    
    
# Grant root write access to our output files 
os.chmod(exp_dir, os.stat(exp_dir).st_mode | ((stat.S_IRWXU | stat.S_IRWXO)))

Start the workflow

In [3]:
wf = pe.Workflow(name='level1', base_dir=exp_dir)
wf.config["execution"]["crashfile_format"] = "txt"

The following two nodes (`infosource` & `dg`) together define all inputs required for the preprocessing workflow

In [4]:
# get subject_id list
subj_list = [x.split('-')[1] for x in glob(data_dir+"sub*")]
subj_list.sort()

In [5]:
infosource = pe.Node(util.IdentityInterface(fields=["subject_id"]),
                  name="infosource")
infosource.iterables = [("subject_id", subj_list)]

In [6]:
dg = pe.Node(
    interface=nio.DataGrabber(
        infields=["subject_id","run_id"], outfields=["struct", "func", "events"]
    ),
    name="dg"
)

# Specify task names and return a sorted filelist to ensure we match files to correct runs/tasks
dg.inputs.run_id = [1,2]
dg.inputs.sort_filelist = True
dg.inputs.template = "*"
dg.inputs.base_directory = data_dir


# Define arguments fill the wildcards in the below paths 
dg.inputs.template_args = dict(
    struct=[["subject_id","subject_id"]],
    func=[["subject_id","subject_id","run_id"]],
    events=[["subject_id","subject_id","run_id"]]
)

# bold – Preprocessed BOLD run spatially normalized to standard space
# mask – Brain mask corresponding to preprocessed BOLD run, in standard space
# events – Original events ﬁle that describes when the subject was exposed to the experimental manipulation
# regressors – Confound signals, a ﬁle corresponding to each BOLD run 

dg.inputs.field_template = dict(
    struct = "sub-%s/anat/sub-%s_T1w.nii.gz",
    func="sub-%s/func/sub-%s_task-flanker_run-%d_bold.nii.gz",
    events="sub-%s/func/sub-%s_task-flanker_run-%d_events.tsv", 
)


wf.connect([
        (infosource, dg, [("subject_id", "subject_id")])
])

## Initialisation

Convert functional images to float representation. Since there can be more than one functional run we use a MapNode to convert each run.

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths ../sub-11/func/sub-11_task-flanker_run-1_bold prefiltered_func_data -odt float
```

In [7]:
img2float = pe.MapNode(
    interface=fsl.ImageMaths(out_data_type='float', op_string='', suffix='_dtype'),
    name='img2float',
    iterfield=['in_file'])
wf.connect(dg, 'func', img2float, 'in_file')

Extract the middle volume of the first run as the reference

(Head movement, motion-correction)

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslroi prefiltered_func_data example_func 73 1
```

In [8]:
# Extract one roi
extract_ref = pe.MapNode(interface=fsl.ExtractROI(t_size=1), 
                         name='extractref',
                         iterfield=['in_file'])

# Define a function to pick the first file from a list of files


wf.connect(img2float, 'out_file', extract_ref, 'in_file')

# Define a function to return the 1 based index of the middle volume

def getmiddlevolume(func):
    from nibabel import load
    funcfile = func
    if isinstance(func, list):
        funcfile = func[0]
    _, _, _, timepoints = load(funcfile).shape
    return int(timepoints/2) 


wf.connect(img2float, ('out_file', getmiddlevolume), extract_ref, 't_min')

## Preprocessing

### Motion Correction

Realign the functional runs to the middle volume of each run

**Corresponding FSL command:**

```
/usr/local/fsl/bin/mcflirt -in prefiltered_func_data -out prefiltered_func_data_mcf -mats -plots -reffile example_func -rmsrel -rmsabs -spline_final
```

```
save_mats (a boolean) – Save transformation matrices. Maps to a command-line argument: -mats.
save_plots (a boolean) – Save transformation parameters. Maps to a command-line argument: -plots.
save_rms (a boolean) – Save rms displacement parameters. Maps to a command-line argument: -rmsabs -rmsrel.
Interpolation (‘spline’ or ‘nn’ or ‘sinc’) – Interpolation method for transformation. Maps to a command-line argument: -%s_final.
```

In [9]:
motion_correct = pe.MapNode(
    interface=fsl.MCFLIRT(save_mats=True, save_plots=True, save_rms=True, interpolation='spline'),
    name='realign',
    iterfield=['in_file','ref_file'])
wf.connect(img2float, 'out_file', motion_correct, 'in_file')
wf.connect(extract_ref, 'roi_file', motion_correct, 'ref_file')

Plot the estimated motion parameters

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fsl_tsplot -i prefiltered_func_data_mcf.par -t 'MCFLIRT estimated rotations (radians)' -u 1 --start=1 --finish=3 -a x,y,z -w 640 -h 144 -o rot.png 

/usr/local/fsl/bin/fsl_tsplot -i prefiltered_func_data_mcf.par -t 'MCFLIRT estimated translations (mm)' -u 1 --start=4 --finish=6 -a x,y,z -w 640 -h 144 -o trans.png 

/usr/local/fsl/bin/fsl_tsplot -i prefiltered_func_data_mcf_abs.rms,prefiltered_func_data_mcf_rel.rms -t 'MCFLIRT estimated mean displacement (mm)' -u 1 -w 640 -h 144 -a absolute,relative -o disp.png
```

In [10]:
plot_motion = pe.MapNode(
    interface=fsl.PlotMotionParams(in_source='fsl'),
    name='plot_motion',
    iterfield=['in_file'])
plot_motion.iterables = ('plot_type', ['rotations', 'translations','displacement'])
wf.connect(motion_correct, 'par_file', plot_motion, 'in_file')

### Functionally Masking

```
Passing the reference volume to the FSL command-line tool bet to generate a binary brain mask and afterward multiplying the processed functional time series by the brain mask using the fslmaths command to produce a skull-stripped time series.
```

See [Ciric et al.(2018)](https://www.nature.com/articles/s41596-018-0065-y#Sec24)

Extract the mean volume of each functional run

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_mcf -Tmean mean_func
```

In [11]:
meanfunc = pe.MapNode(
    interface=fsl.ImageMaths(op_string='-Tmean', suffix='_mean'),
    name='meanfunc',
    iterfield=['in_file'])
wf.connect(motion_correct, 'out_file', meanfunc, 'in_file')

Strip the skull from the mean functional to generate a mask

**Corresponding FSL command:**

```
/usr/local/fsl/bin/bet2 mean_func mask -f 0.3 -n -m; /usr/local/fsl/bin/immv mask_mask mask
```

In [12]:
meanfuncmask = pe.MapNode(
    interface=fsl.BET(mask=True, no_output=True, frac=0.3),
    name='meanfuncmask',
    iterfield=['in_file'])
wf.connect(meanfunc, 'out_file', meanfuncmask, 'in_file')

Mask the functional data with the extracted mask

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_mcf -mas mask prefiltered_func_data_bet
```

In [13]:
maskfunc = pe.MapNode(
    interface=fsl.ImageMaths(suffix='_bet', op_string='-mas'),
    name='maskfunc',
    iterfield=['in_file','in_file2'])
wf.connect(motion_correct, 'out_file', maskfunc, 'in_file')
wf.connect(meanfuncmask, 'mask_file', maskfunc, 'in_file2')

### Grand Mean Scaling

Determine the 2nd and 98th percentile intensities 

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslstats prefiltered_func_data_bet -p 2 -p 98
0.000000 873.492249 (these numbers are for subject-11 run-01)
```

More info [here](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Fslutils#fslstats)

In [14]:
getthresh = pe.MapNode(
    interface=fsl.ImageStats(op_string='-p 2 -p 98'),
    name='getthreshold',
    iterfield=['in_file'])
wf.connect(maskfunc, 'out_file', getthresh, 'in_file')

Threshold the first TR of the functional data at 10% of the 98th percentile

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_bet -thr 87.3492249 -Tmin -bin mask -odt char
```

In [15]:
threshold = pe.MapNode(
    interface=fsl.ImageMaths(out_data_type='char', suffix='_thresh'),
    name='threshold',
    iterfield=['in_file','op_string'])
wf.connect(maskfunc, 'out_file', threshold, 'in_file')

# Define a function to get 10% of the intensity
def getthreshop(thresh):
    
    return ['-thr %.10f -Tmin -bin' % (0.1 * th[1]) for th in thresh]

wf.connect(getthresh, ('out_stat', getthreshop), threshold, 'op_string')

Determine the median value of the TRs using the mask

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslstats prefiltered_func_data_mcf -k mask -p 50
728.800232 (this number is for subject-11 run-01)
```

In [16]:
medianval = pe.MapNode(
    interface=fsl.ImageStats(op_string='-k %s -p 50'),
    name='medianval',
    iterfield=['in_file','mask_file'])
wf.connect(motion_correct, 'out_file', medianval, 'in_file')
wf.connect(threshold, 'out_file', medianval, 'mask_file')

Dilate the mask

The brain mask is "dilated" slightly before being used. Because it is normally important that masking be liberal (ie that there be little risk of cutting out valid brain voxels) 

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths mask -dilF mask
```

The output of `dilatemask` (i.e., `/srv/scratch/yc/fsl/hw2/level1/_subject_id_26/dilatemask/mapflow/_dilatemask0/sub-26_task-flanker_run-1_bold_dtype_mcf_bet_thresh_dil.nii.gz`) is equivalent to `mask.nii.gz` from FSL GUI.

In [17]:
dilatemask = pe.MapNode(
    interface=fsl.ImageMaths(suffix='_dil', op_string='-dilF'),
    name='dilatemask',
    iterfield=['in_file'])
wf.connect(threshold, 'out_file', dilatemask, 'in_file')

Mask the motion corrected functional runs with the dilated mask

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_mcf -mas mask prefiltered_func_data_thresh
```

In [18]:
maskfunc2 = pe.MapNode(
    interface=fsl.ImageMaths(suffix='_thresh', op_string='-mas'),
    iterfield=['in_file','in_file2'],
    name='maskfunc2')
wf.connect(motion_correct, 'out_file', maskfunc2, 'in_file')
wf.connect(dilatemask, 'out_file', maskfunc2, 'in_file2')

### SUSAN Noise Reduction

Determine the mean image from each TR

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_thresh -Tmean mean_func
```

In [19]:
meanfunc2 = pe.MapNode(
    interface=fsl.ImageMaths(op_string='-Tmean', suffix='_mean'),
    name='meanfunc2',
    iterfield=['in_file'])
wf.connect(maskfunc2, 'out_file', meanfunc2, 'in_file')

Merge the median values with the mean functional images into a coupled list

The output of this `merge` node will go into `susan` as `usans`,

In [20]:
# why here use node not mapnode?
mergenode = pe.Node(interface=util.Merge(2, axis='hstack'), 
                       name='merge')
wf.connect(meanfunc2, 'out_file', mergenode, 'in1')
wf.connect(medianval, 'out_stat', mergenode, 'in2')

Smooth each run using SUSAN with the brightness threshold set to 75% of the median value for each run and a mask constituting the mean functional

```
Usage: susan <input> <bt> <dt> <dim> <use_median> <n_usans> [<usan1> <bt1> [<usan2> <bt2>]] <output>
<bt> is brightness threshold and should be greater than noise level and less than contrast of edges to be preserved.
<dt> is spatial size (sigma, i.e., half-width) of smoothing, in mm.
<dim> is dimensionality (2 or 3), depending on whether smoothing is to be within-plane (2) or fully 3D (3).
<use_median> determines whether to use a local median filter in the cases where single-point noise is detected (0 or 1).
<n_usans> determines whether the smoothing area (USAN) is to be found from secondary images (0, 1 or 2).
A negative value for any brightness threshold will auto-set the threshold at 10% of the robust range
```

**Corresponding FSL command:**

```
/usr/local/fsl/bin/susan prefiltered_func_data_thresh 546.600174 2.12314225053 3 1 1 mean_func 546.600174 prefiltered_func_data_smooth
```

**Note:**

for `<bt>`, Nipype uses a different algorithm to calculate it -> `float(fwhm) / np.sqrt(8 * np.log(2))`. Therefore, to get `2.12314225053`, fwhm should be `4.9996179300001655` instead of `5`

In [21]:
fwhm_thr = 4.9996179300001655

smooth = pe.MapNode(
    interface=fsl.SUSAN(fwhm = fwhm_thr),
    name='smooth',
    iterfield=['in_file', 'brightness_threshold', 'usans'])

# Define a function to get the brightness threshold for SUSAN
def getbtthresh(medianvals):
    return [0.75 * val for val in medianvals]


def getusans(x):
    return [[tuple([val[0], 0.75 * val[1]])] for val in x]


wf.connect(maskfunc2, 'out_file', smooth, 'in_file')
wf.connect(medianval, ('out_stat', getbtthresh), smooth,
                'brightness_threshold')
wf.connect(mergenode, ('out', getusans), smooth, 'usans')

Mask the smoothed data with the dilated mask


**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_smooth -mas mask prefiltered_func_data_smooth
```

In [22]:
maskfunc3 = pe.MapNode(
    interface=fsl.ImageMaths(op_string='-mas'),
    name='maskfunc3',
    iterfield=['in_file','in_file2'])
wf.connect(smooth, 'smoothed_file', maskfunc3, 'in_file')
wf.connect(dilatemask, 'out_file', maskfunc3, 'in_file2')

Scale each volume of the TR so that the median value of the TR is set to 10000

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_smooth -mul 13.7211811425 prefiltered_func_data_intnorm 
(this number is for subject-11 run-01)
```

In [23]:
intnorm = pe.MapNode(
    interface=fsl.ImageMaths(suffix='_intnorm'),
    iterfield=['in_file', 'op_string'],
    name='intnorm')
wf.connect(maskfunc3, 'out_file', intnorm, 'in_file')

# Define a function to get the scaling factor for intensity normalization
def getinormscale(medianvals):
    return ['-mul %.10f' % (10000. / val) for val in medianvals]


wf.connect(medianval, ('out_stat', getinormscale), intnorm, 'op_string')

### Temporal Filtering

Perform temporal highpass filtering on the data

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_intnorm -Tmean tempMean
/usr/local/fsl/bin/fslmaths prefiltered_func_data_intnorm -bptf 25.0 -1 -add tempMean prefiltered_func_data_tempfilt
```

The output of `highpass` (i.e., `sub-11_task-flanker_run-1_bold_dtype_mcf_mask_smooth_mask_intnorm_tempfilt.nii.gz`) is equivalent to `filtered_func_data.nii.gz` from FSL GUI.

In [24]:
# Generate a mean functional image from the scaled data and this mean func will be used in performing temporal highpass filtering
meanfunc3 = pe.MapNode(
    interface=fsl.ImageMaths(op_string='-Tmean'),
    iterfield=['in_file'],
    name='meanfunc3')
wf.connect(intnorm, 'out_file', meanfunc3, 'in_file')

In [25]:
# Perform temporal highpass filtering on the data
hpcutoff = 100
TR = 2.  # ensure float

highpass = pe.MapNode(
    interface=fsl.ImageMaths(suffix='_tempfilt'),
    name='highpass',
    iterfield=['in_file','op_string'])

# 25 = (hpcutoff / 2*TR) not (hpcutoff / TR)
def gethpstring(tempMean):
    return ['-bptf 25 -1 -add %s' % (tm) for tm in tempMean]

wf.connect(intnorm, 'out_file', highpass, 'in_file')
wf.connect(meanfunc3, ('out_file',gethpstring), highpass, 'op_string')


Generate a mean functional image from the functional run

**Corresponding FSL command:**

```
/usr/local/fsl/bin/fslmaths prefiltered_func_data_tempfilt filtered_func_data
/usr/local/fsl/bin/fslmaths filtered_func_data -Tmean mean_func
```

In [26]:
meanfunc4 = pe.MapNode(
    interface=fsl.ImageMaths(op_string='-Tmean', suffix='_mean'),
    iterfield=['in_file'],
    name='meanfunc4')
wf.connect(highpass, 'out_file', meanfunc4, 'in_file')

## First-Level GLM

### Get events

In [27]:
def subjinfo(events):
    from nipype.interfaces.base import Bunch
    import pandas as pd
    import numpy as np

    subject_info = []

    ev = pd.read_csv(events, sep="\t")
    ev = ev[ev['correctness']=='correct']
    ev['new_type'] = ev['trial_type'].apply(lambda x: str(x).split('_')[0])

    run_info = Bunch(onsets=[], 
                     durations=[])

    run_info.set(conditions=[g[0] for g in ev.groupby("new_type")])

    for group in ev.groupby("new_type"):
        run_info.onsets.append(group[1].onset.tolist())
        run_info.durations.append(group[1].duration.tolist())
    subject_info.append(run_info)

    return subject_info

get_sub_info = pe.MapNode(
    Function(
        function=subjinfo, input_names=["events"], output_names="subject_info"
    ),
    name="get_sub_info", iterfield=["events"]
)

# Connect to workflow
wf.connect(dg, 'events', get_sub_info, 'events')


### Set-up contrasts

In [28]:
condition_names = ["congruent", "incongruent"]

# Activation Contrasts (similar to https://direct.mit.edu/jocn/article/23/10/3162/5327/Is-Morality-Unified-Evidence-that-Distinct-Neural)
## From FEAT: "The correct way to tell whether two conditions are significantly different is to run a differential contrast like [1 -1] between them"
cont01 = ["congruent", "T", condition_names,  [1, 0]]
cont02 = ["incongruent", "T", condition_names, [0, 1]]
cont03 = ["congruent-incongruent", "T", condition_names, [1, -1]]
cont04 = ["incongruent-congruent", "T", condition_names,  [-1, 1]]

contrast_list = [
    cont01,
    cont02,
    cont03,
    cont04
]

In [29]:
TR = 2.0  # Repetition Time
hpcutoff = 100  # highpass filter cutoff in seconds! 

modelspec = pe.MapNode(interface=model.SpecifyModel(), name="modelspec", iterfield=["subject_info","functional_runs"])
modelspec.inputs.input_units = "secs"
modelspec.inputs.high_pass_filter_cutoff = hpcutoff
modelspec.inputs.time_repetition = TR

wf.connect(get_sub_info, 'subject_info', modelspec,'subject_info')
wf.connect(highpass, 'out_file',modelspec,'functional_runs')

In [30]:
level1design = pe.MapNode(interface=fsl.Level1Design(), name="level1design", iterfield=["session_info"])
level1design.inputs.interscan_interval = TR
# Set HRF bases functions
level1design.inputs.bases = {"gamma": {"derivs": False, 'gammasigma':3, 'gammadelay':6}}
level1design.inputs.model_serial_correlations = True
level1design.inputs.contrasts = contrast_list

wf.connect(modelspec,'session_info', level1design, 'session_info')

In [31]:
modelgen = pe.MapNode(
    interface=fsl.FEATModel(),
    name='modelgen',
    iterfield=['fsf_file', 'ev_files'])

wf.connect([(level1design, modelgen, [('fsf_files', 'fsf_file'), 
                                    ('ev_files','ev_files')])])

**Corresponding fsl command:**

```
/usr/local/fsl/bin/film_gls --in=filtered_func_data --rn=stats --pd=design.mat --thr=1000.0 --sa --ms=5 --con=design.con  

--thr: threshold
--sa: smooth_autocorr
--ms: mask_size
```

In [32]:
level1estimate = pe.MapNode(
    interface=fsl.FILMGLS(smooth_autocorr=True, mask_size=5, threshold=1000),
    name='level1estimate',
    iterfield=['design_file', 'in_file', 'tcon_file'])

wf.connect([
    (highpass,level1estimate,[('out_file','in_file')]),
    (modelgen,level1estimate,[('design_file','design_file'),
                               ('con_file','tcon_file')])
])

no need to use `ContrastMgr`, 

```
In interface mode this file assumes that all the required inputs are in the same location. This has deprecated for FSL versions 5.0.7+ as the necessary corrections file is no longer generated by FILMGLS.
```
see this [link](https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.fsl.model.html#contrastmgr)

## Registration

According to [FSL UserGuide](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FEAT/UserGuide#Higher-Level_FEAT_Output), the different sessions need to be **registered** to each other before any multi-session or multi-subject analyses can be carried out. Registration inside FEAT uses `FLIRT` and is a two-statge process:

1. An example FMRI low resolution image is registered to an example high resolution image (normally the same subject's T1-weighted structural). The transformation for this is saved into the FEAT directory. Then the high res image is registered to a standard image (normally a T1-weighted image in standard space, such as the MNI 152 average image).
2. The two transformations are combined into a third, which will take the low resolution FMRI images (and the statistic images derived from the first-level analyses) straight into standard space, when applied later, during group analysis.

#### Step 1
**Corresponding FSL command:**

```
/usr/local/fsl/bin/flirt -in example_func -ref standard -out example_func2standard -omat example_func2standard.mat -cost corratio -dof 12 -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -interp trilinear 

/usr/local/fsl/bin/convert_xfm -inverse -omat standard2example_func.mat example_func2standard.mat
```

In [33]:
flt = pe.MapNode(interface=fsl.FLIRT(cost='corratio', dof=12,
                                    searchr_x = [-90,90],
                                    searchr_y = [-90,90],
                                    searchr_z = [-90,90],
                                    interp = 'trilinear'), 
               name='example_func2standard',
               iterfield=['in_file'])

flt.inputs.reference = fsl.Info.standard_image('MNI152_T1_2mm_brain.nii.gz')
wf.connect(extract_ref, 'roi_file', flt, 'in_file')

convertxfm = pe.MapNode(interface=fsl.ConvertXFM(invert_xfm = True), 
                         name='convertxfm',
                         iterfield=['in_file'])
wf.connect(flt, 'out_matrix_file', convertxfm, 'in_file')

#### Step 2

**Corresponding fsl command:**

```
/usr/local/fsl/bin/flirt -ref reg/standard -in stats/cope1 -out /srv/scratch/yc/fsl/nipype_fsl_comp/gui/sub-01/run1.feat/frgrot_chksugax -applyxfm -init reg/example_func2standard.mat -interp trilinear -datatype float
```

In [34]:
def warp_files(copes, varcopes, masks, mat):
    # need to reimport here, otherwise errors come out
    import nipype.interfaces.fsl as fsl 
    
    out_copes = []
    out_varcopes = []
    out_masks = []
    
    # register mask, same function, different parameters
    warp_mask = fsl.FLIRT(apply_xfm = True, 
                     interp = 'nearestneighbour')
    warp_mask.inputs.reference = fsl.Info.standard_image('MNI152_T1_2mm_brain.nii.gz')
    warp_mask.inputs.in_matrix_file = mat
    warp_mask.inputs.output_type = "NIFTI_GZ"
    warp_mask.inputs.in_file = masks
    res_mask = warp_mask.run()
    out_masks.append(str(res_mask.outputs.out_file))
    
    # register copes & varcopes using same function, different parameters
    warp = fsl.FLIRT(apply_xfm = True, 
                     interp = 'trilinear')
    warp.inputs.reference = fsl.Info.standard_image('MNI152_T1_2mm_brain.nii.gz')
    warp.inputs.in_matrix_file = mat
    warp.inputs.output_type = "NIFTI_GZ"
    
    
    # register copes
    for cope in copes:
        warp.inputs.in_file = cope
        res = warp.run()
        out_copes.append(str(res.outputs.out_file))
        
     # register varcopes
    for varcope in varcopes:
        warp.inputs.in_file = varcope
        res = warp.run()
        out_varcopes.append(str(res.outputs.out_file))

    return out_copes, out_varcopes, out_masks

In [35]:
warpfunc = pe.MapNode(util.Function(input_names=['copes', 'varcopes', 'masks', 'mat'],
                                    output_names=['out_copes', 'out_varcopes', 'out_masks'],
                                    function=warp_files),
                      iterfield=['copes', 'varcopes', 'masks','mat'],
                      name='warpfunc')

wf.connect(level1estimate, 'copes', warpfunc, 'copes')
wf.connect(level1estimate, 'varcopes', warpfunc, 'varcopes')
wf.connect(dilatemask, 'out_file', warpfunc, 'masks')
wf.connect(flt, 'out_matrix_file', warpfunc, 'mat')

In [36]:
# save all results into one

datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(exp_dir, "level1_results")

wf.connect(infosource, 'subject_id', datasink, 'container')
wf.connect([(level1estimate, datasink, [('results_dir', 'results_dir')])])
wf.connect([(warpfunc, datasink, [('out_copes', 'reg_copes')])])
wf.connect([(warpfunc, datasink, [('out_varcopes', 'reg_varcopes')])])
wf.connect([(warpfunc, datasink, [('out_masks', 'reg_masks')])])

In [ ]:
# Create 1st-level analysis output graph
wf.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=os.path.join(wf.base_dir, wf.name, 'graph.png'))


In [ ]:
# Run Workflow
wf.run(plugin="MultiProc", plugin_args={"n_procs": 8})

220124-00:33:45,769 nipype.workflow INFO:
	 Workflow level1 settings: ['check', 'execution', 'logging', 'monitoring']
220124-00:33:46,236 nipype.workflow INFO:
	 Running in parallel.
220124-00:33:46,248 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 26 jobs ready. Free memory (GB): 27.96/27.96, Free processors: 8/8.
220124-00:33:46,325 nipype.workflow INFO:
	 [Node] Setting-up "level1.dg" in "/home/yc/out/fsl/hw2/level1/_subject_id_26/dg".
220124-00:33:46,332 nipype.workflow INFO:
	 [Node] Setting-up "level1.dg" in "/home/yc/out/fsl/hw2/level1/_subject_id_21/dg".
220124-00:33:46,333 nipype.workflow INFO:
	 [Node] Setting-up "level1.dg" in "/home/yc/out/fsl/hw2/level1/_subject_id_20/dg".
220124-00:33:46,334 nipype.workflow INFO:
	 [Node] Setting-up "level1.dg" in "/home/yc/out/fsl/hw2/level1/_subject_id_19/dg".
220124-00:33:46,340 nipype.workflow INFO:
	 [Node] Running "dg" ("nipype.interfaces.io.DataGrabber")
220124-00:33:46,345 nipype.workflow INFO:
	 [Node] Running "dg" ("n